In [9]:
from IPython.display import Image, display
%matplotlib inline
from networkx.readwrite import json_graph
import graphviz as gv

import warnings
warnings.filterwarnings("ignore")

In [10]:
styles = {
    'graph': {
        'fontsize': '16',
        'fontcolor': 'black',
        'bgcolor': '#ffffff',
    },
    'nodes': {
        'fontname': 'Helvetica',
        'shape': 'Mdiamond',
        'fontcolor': '#78909C',
        'color': 'white',
        'style': 'filled',
        'fillcolor': '#ECEFF1',
    },
    'edges': {
        'style': 'solid',
        'color': 'black',
        'arrowhead': 'open',
        'fontname': 'Helvetica',
        'fontsize': '12',
        'fontcolor': '#78909C',
    }
}

In [11]:
def apply_styles(g, styles):
    g.graph_attr.update(
        styles['graph']
    )
    g.node_attr.update(
        ('nodes' in styles and styles['nodes']) or {}
    )
    g.edge_attr.update(
        ('edges' in styles and styles['edges']) or {}
    )
    return g

In [29]:
def save_graph(recipe, directory="./recipe_graphs"):
    g = json_graph.node_link_graph(recipe["steps"])
    G = gv.Digraph(format='png', graph_attr={'label': recipe["title"]})
    nodes = g.nodes(data=True)
    
    for node in nodes:
        t = node[1]["type"]
        name = node[1]["label"]
        if t == "heat":
            if name == "very high":
                G.node(str(node[0]), name + " heat", fontcolor='#263238', fillcolor='#FF5542', shape='diamond')
            elif name == "high":
                G.node(str(node[0]), name + " heat", fontcolor='#263238', fillcolor='#FF7043', shape='diamond')
            elif name == "medium":
                G.node(str(node[0]), name + " heat", fontcolor='#263238', fillcolor='#FFA726', shape='diamond')
            elif name == "low":
                G.node(str(node[0]), name + " heat", fontcolor='#263238', fillcolor='#FFCA28', shape='diamond')
            elif name == "no":
                G.node(str(node[0]), name + " heat", fontcolor='#212121', fillcolor='#90A4AE', shape='diamond')
        elif t == "ingredient":
            descript = recipe["ingredients"][name]["descript"]
            label = ""
            for d in descript:
                label += d + "\n"
            label += name
            G.node(str(node[0]), label)
        elif t == "done":
            G.node(str(node[0]), name, fontcolor='black', fillcolor='#66BB6A', shape='egg')
    
    for edge in g.edges():
        for node in nodes:
            if node[0] == edge[0]:
                if node[1]["type"] == "ingredient":
                    name = node[1]["label"]
                    ingredient = recipe['ingredients'][name]
                    amount = ""
                    if ingredient['amount'] > 0:
                        amount += "%(amt).1f" % {'amt': ingredient['amount']}
                        if ingredient['unit'] != None:
                            amount += " %(unit)s" % {'unit': ingredient['unit']}
                    if amount != "":
                        nodeID = "%(name)s amt" % {'name': name}
                        G.node(nodeID, amount, shape='ellipse')
                        G.edge(nodeID, str(edge[0]), style='dashed', arrowhead='none')
        G.edge(str(edge[0]), str(edge[1]))
        
    G = apply_styles(G, styles)
            
    filename = "%(directory)s/%(title)s" % {'directory': directory, 'title': recipe["title"].strip()} # format type=png set above
    G.render(filename=filename, cleanup=True)
    return filename + ".png"

In [13]:
def show_graph(path):    
    display(Image(filename=path))

In [ ]:
def prettyprint_recipe(recipe): #TODO
    #generate text
    #generate title
    pass